In [58]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# Parámetros de conexión (modifica con tus datos)
db_user = 'arq'
db_password = 'password'
db_host = 'localhost'
db_port = '5432'
db_name = 'final_gya'

# Crear el engine de conexión
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [59]:
# Cargar las tablas en dataframes
df_ledger = pd.read_sql_table('ledger', engine, schema='esquemagya')
df_facturas = pd.read_sql_table('hechos_facturas', engine, schema='esquemagya')
df_o2c_review_level = pd.read_sql_table('o2c_review_level', engine, schema='esquemagya')

# Visualizamos las primeras filas de cada tabla
print("Ledger:")
print(df_ledger.head())
print("\nFacturas:")
print(df_facturas.head())
print("\nReview Level:")
print(df_o2c_review_level.head())

Ledger:
  region  county    city  tax_collected
0     GA   Bacon    Alma            0.0
1     GA   Bacon    Alma          330.9
2     GA   Bacon    Alma            0.0
3     GA   Bacon    Alma            0.0
4     GA  Barrow  Auburn            8.4

Facturas:
  facturas_durolast facturas_fsbp facturas_ses facturas_negatives  \
0              None      91569359         None               None   
1              None      91381448         None               None   
2              None      91395643         None               None   
3              None      91373071         None               None   
4              None      91373070         None               None   

  facturas_gaco state                  city  county  total_tax_amount fuente  
0          None    CO              Longmont    Weld            -331.3   FSBP  
1          None    GA  Unincorporated Areas  Coffee           -9150.8   FSBP  
2          None    GA  Unincorporated Areas  Coffee           -8449.0   FSBP  
3         

In [60]:
def extraer_numero_factura(row):
    # Revisamos cada columna en el orden dado
    for col in ['facturas_durolast', 'facturas_ses', 'facturas_negatives', 'facturas_gaco', 'facturas_fsbp']:
        if pd.notnull(row[col]):
            return row[col]
    return np.nan

# Aplicar la función para crear una nueva columna 'invoice_number'
df_facturas['invoice_number'] = df_facturas.apply(extraer_numero_factura, axis=1)

# Verificamos la nueva columna
print(df_facturas[['facturas_durolast', 'facturas_ses', 'facturas_negatives', 'facturas_gaco', 'facturas_fsbp', 'invoice_number']].head())


  facturas_durolast facturas_ses facturas_negatives facturas_gaco  \
0              None         None               None          None   
1              None         None               None          None   
2              None         None               None          None   
3              None         None               None          None   
4              None         None               None          None   

  facturas_fsbp invoice_number  
0      91569359       91569359  
1      91381448       91381448  
2      91395643       91395643  
3      91373071       91373071  
4      91373070       91373070  


In [61]:
# Hacemos un merge (join) con la tabla review para agregar el review_level con base en el estado
df_ledger = df_ledger.merge(df_o2c_review_level, how='left', left_on='region', right_on='state')
df_facturas = df_facturas.merge(df_o2c_review_level, how='left', left_on='state', right_on='state')

# Verificamos la incorporación
print("Ledger con review level:")
print(df_ledger.head(), "\n")

print("Facturas con review level:")
print(df_facturas[['state', 'county', 'city', 'o2c_review_level', 'invoice_number', 'total_tax_amount']].head())


Ledger con review level:
  region  county    city  tax_collected state o2c_review_level
0     GA   Bacon    Alma            0.0    GA           County
1     GA   Bacon    Alma          330.9    GA           County
2     GA   Bacon    Alma            0.0    GA           County
3     GA   Bacon    Alma            0.0    GA           County
4     GA  Barrow  Auburn            8.4    GA           County 

Facturas con review level:
  state  county                  city o2c_review_level invoice_number  \
0    CO    Weld              Longmont             City       91569359   
1    GA  Coffee  Unincorporated Areas           County       91381448   
2    GA  Coffee  Unincorporated Areas           County       91395643   
3    GA  Coffee  Unincorporated Areas           County       91373071   
4    GA  Coffee  Unincorporated Areas           County       91373070   

   total_tax_amount  
0            -331.3  
1           -9150.8  
2           -8449.0  
3           -6606.9  
4           -6606.9

In [62]:
def obtener_llave_agrupamiento(review_level):
    if review_level.lower() == 'state':
        return ['state']  # o 'region' según corresponda, pero usaremos state
    elif review_level.lower() == 'county':
        return ['state', 'county']
    elif review_level.lower() == 'city' or review_level.lower() == 'parish' or review_level.lower() == 'bourough':
        return ['state', 'county', 'city']
    else:
        raise ValueError(f"Nivel de revisión desconocido: {review_level}")

In [63]:
# Inicializamos una lista vacía para almacenar los resultados finales
resultados = []  # Aquí se guardarán los resultados finales

# Obtenemos los estados únicos del DataFrame 'df_o2c_review_level'
estados = df_o2c_review_level['state'].unique()

# Iteramos sobre cada estado para procesar la información de forma separada
for estado in estados:
    # Obtenemos el nivel de revisión ('review_level') correspondiente al estado actual
    review_level = df_o2c_review_level.loc[df_o2c_review_level['state'] == estado, 'o2c_review_level'].iloc[0]
    
    # Determinamos la llave de agrupamiento basada en el 'review_level'
    llave = obtener_llave_agrupamiento(review_level)
    
    # Filtramos los datos del estado en los DataFrames correspondientes
    df_ledger_estado = df_ledger[df_ledger['region'] == estado].copy()
    df_facturas_estado = df_facturas[df_facturas['state'] == estado].copy()
    
    # Agrupamos el recaudo de impuestos según la llave y sumamos los valores
    recaudado = df_ledger_estado.groupby(llave)['tax_collected'].sum().reset_index()
    
    # Seleccionamos las columnas relevantes en las facturas
    facturas_grouped = df_facturas_estado[llave + ['invoice_number', 'total_tax_amount', 'fuente']]
    
    # Agrupamos las facturas según la llave
    grupos = facturas_grouped.groupby(llave)
    
    # Iteramos sobre cada grupo de facturas
    for llave_valores, df_grupo in grupos:
        # Normalizamos la llave en un diccionario
        key_dict = dict(zip(llave, llave_valores)) if isinstance(llave_valores, tuple) else {llave[0]: llave_valores}
        
        # Filtramos el monto recaudado correspondiente al grupo actual
        recaudado_row = recaudado.copy()
        for col, val in key_dict.items():
            recaudado_row = recaudado_row[recaudado_row[col] == val]
        
        # Asignamos el monto recaudado o 0 si no hay datos
        fondos_disponibles = recaudado_row['tax_collected'].iloc[0] if not recaudado_row.empty else 0
        
        # Ordenamos las facturas por valor absoluto del monto de mayor a menor
        df_grupo_sorted = df_grupo.sort_values(by='total_tax_amount', ascending=True, key=abs)
        
        # Seleccionamos las facturas cuya suma se aproxime más a los fondos disponibles
        mejor_seleccion = []
        mejor_diferencia = fondos_disponibles
        
        for i in range(len(df_grupo_sorted)):
            seleccion_actual = []
            suma_actual = 0
            for _, factura in df_grupo_sorted.iloc[i:].iterrows():
                costo = abs(factura['total_tax_amount'])
                if suma_actual + costo <= fondos_disponibles:
                    suma_actual += costo
                    seleccion_actual.append((factura['invoice_number'], factura['fuente']))
                if suma_actual == fondos_disponibles:
                    break  # Si alcanzamos exactamente el monto disponible, terminamos
            
            diferencia_actual = fondos_disponibles - suma_actual
            if diferencia_actual < mejor_diferencia:
                mejor_diferencia = diferencia_actual
                mejor_seleccion = seleccion_actual
                
                # Si encontramos la combinación óptima, salimos del loop
                if mejor_diferencia == 0:
                    break
        
        # Guardamos las facturas seleccionadas en los resultados
        for factura_info in mejor_seleccion:
            invoice_num, fuente = factura_info
            registro = key_dict.copy()
            registro['invoice_number'] = invoice_num
            registro['fuente'] = fuente.lower()
            registro['o2c_review_level'] = review_level.lower()
            resultados.append(registro)

# Convertimos la lista de resultados en un DataFrame
df_resultado = pd.DataFrame(resultados)

# Imprimimos los resultados
print("Facturas pagables:")
print(df_resultado)


Facturas pagables:
   state       county                  city invoice_number     fuente  \
0     AL   Tuscaloosa             Northport         737582  duro last   
1     KS      Shawnee                Topeka         761842  duro last   
2     MS       Rankin                   NaN       91845367       fsbp   
3     MS       Rankin                   NaN       91803934       fsbp   
4     MS       Rankin                   NaN       91830724       fsbp   
5     MS       Rankin                   NaN       91842505       fsbp   
6     MS       Rankin                   NaN       91798188       fsbp   
7     MS       Rankin                   NaN       91798187       fsbp   
8     MO       Greene           Springfield         754423  duro last   
9     NE         Hall          Grand Island       91875507       fsbp   
10    NY     Dutchess  Unincorporated Areas       91819546       fsbp   
11    NY     Dutchess  Unincorporated Areas       91819547       fsbp   
12    NY  Westchester           

In [57]:
df_resultado.to_csv('facturas_pagables.csv', index=False)